# Memoria

Por padrao, cada chamada a um agente e independente. O agente nao tem acesso ao que foi dito em chamadas anteriores. Isso significa que, se voce disser seu nome em uma mensagem e perguntar em outra, ele nao vai lembrar.

Para que o agente mantenha o contexto entre interacoes, precisamos adicionar um mecanismo de **memoria**. No LangChain, isso e feito por meio de um **checkpointer**, que salva o historico de mensagens e o associa a uma sessao (thread).

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## Sem memoria

Vamos comecar criando um agente simples, sem nenhum mecanismo de persistencia, e observar o que acontece quando tentamos ter uma conversa em multiplas chamadas.

In [ ]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage

agente = create_agent(model="gpt-4.1-nano")

In [ ]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Ola, meu nome e Carlos e trabalho com inteligencia artificial.")]}
)

print(resposta["messages"][-1].content)

In [ ]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Qual e o meu nome e com o que eu trabalho?")]}
)

print(resposta["messages"][-1].content)

O agente nao lembra. Cada chamada `invoke` e tratada como uma conversa completamente nova. Nao ha nenhum registro do que foi dito antes.

## Com memoria

Para adicionar memoria, usamos o `InMemorySaver` como checkpointer. Ele armazena o historico de mensagens na memoria RAM durante a execucao.

Alem do checkpointer, precisamos identificar cada conversa com um `thread_id`. Isso permite que o agente mantenha multiplas conversas independentes ao mesmo tempo, cada uma com seu proprio historico.

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

agente = create_agent(
    model="gpt-4.1-nano",
    checkpointer=InMemorySaver()
)

Agora, em toda chamada `invoke`, vamos passar um `config` com o `thread_id`. Todas as chamadas com o mesmo `thread_id` compartilham o mesmo historico.

In [ ]:
config = {"configurable": {"thread_id": "conversa-1"}}

In [ ]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Ola, meu nome e Carlos e trabalho com inteligencia artificial.")]},
    config
)

print(resposta["messages"][-1].content)

In [ ]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Qual e o meu nome e com o que eu trabalho?")]},
    config
)

print(resposta["messages"][-1].content)

Agora o agente lembra do que foi dito. O `InMemorySaver` armazenou as mensagens da primeira chamada e as disponibilizou automaticamente na segunda, porque ambas usam o mesmo `thread_id`.

## Threads independentes

O `thread_id` funciona como um identificador de sessao. Threads diferentes sao conversas completamente isoladas. Isso e essencial em aplicacoes reais, onde cada usuario precisa ter sua propria conversa.

In [ ]:
config_outra = {"configurable": {"thread_id": "conversa-2"}}

resposta = agente.invoke(
    {"messages": [HumanMessage(content="Qual e o meu nome?")]},
    config_outra
)

print(resposta["messages"][-1].content)

Na `conversa-2`, o agente nao sabe o nome do usuario porque e uma thread diferente. Cada thread tem seu proprio historico, completamente independente das outras.

In [ ]:
resposta = agente.invoke(
    {"messages": [HumanMessage(content="Qual e o meu nome?")]},
    config
)

print(resposta["messages"][-1].content)

Voltando para a `conversa-1`, o agente ainda lembra de tudo.

Na pratica, o `InMemorySaver` e ideal para prototipacao e testes. Para aplicacoes em producao, o LangGraph oferece checkpointers que persistem os dados em bancos de dados como PostgreSQL ou SQLite, garantindo que o historico sobreviva a reinicializacoes do servidor.